In [ ]:
import numpy as np
import ufl
from mpi4py import MPI
from dolfinx import mesh, fem, io
from petsc4py import PETSc
from slepc4py import SLEPc

# Geometry and mesh
L, W, H = 1.0, 0.1, 0.1  # dimensions in meters
Nx, Ny, Nz = 10, 2, 2    # mesh resolution

domain = mesh.create_box(
    MPI.COMM_WORLD,
    [np.array([0, 0, 0]), np.array([L, W, H])],
    [Nx, Ny, Nz],
    cell_type=mesh.CellType.hexahedron
)

# Define function space
V = fem.VectorFunctionSpace(domain, ("Lagrange", 1))

# Material properties
E = 2.1e11       # Young's modulus (Pa)
nu = 0.3         # Poisson's ratio
rho = 7800       # Density (kg/m³)

mu = E / (2 * (1 + nu))
lmbda = E * nu / ((1 + nu) * (1 - 2 * nu))

def sigma(u):
    return 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * ufl.Identity(3)

# Trial and test functions
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Bilinear forms
a_form = ufl.inner(sigma(u), ufl.grad(v)) * ufl.dx
m_form = rho * ufl.inner(u, v) * ufl.dx

# Dirichlet boundary condition: fix left face (x=0)
def left_boundary(x):
    return np.isclose(x[0], 0.0)

bc = fem.dirichletbc(PETSc.ScalarType(0.0),
                     fem.locate_dofs_geometrical(V, left_boundary), V)

# Assemble matrices
a = fem.form(a_form)
m = fem.form(m_form)

A = fem.petsc.assemble_matrix(a, bcs=[bc])
A.assemble()
M = fem.petsc.assemble_matrix(m)
M.assemble()

# Solve generalized eigenvalue problem
eigensolver = SLEPc.EPS().create()
eigensolver.setOperators(A, M)
eigensolver.setProblemType(SLEPc.EPS.ProblemType.GHEP)
eigensolver.setDimensions(nev=6)
eigensolver.solve()

# Extract results
n_conv = eigensolver.getConverged()
u = fem.Function(V)

print("\nNatural frequencies (Hz):")
frequencies = []

for i in range(n_conv):
    eig_val = eigensolver.getEigenvalue(i)
    omega = np.sqrt(np.abs(eig_val.real))
    freq = omega / (2 * np.pi)
    frequencies.append(freq)
    print(f"Mode {i+1}: {freq:.2f} Hz")

# Export mode shapes
with io.XDMFFile(domain.comm, "modes.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    for i in range(n_conv):
        eig_vec = eigensolver.getEigenvector(i)
        u.vector.setArray(eig_vec)
        u.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
        xdmf.write_function(u, i)

print("\nMode shapes saved to 'modes.xdmf'. Open in ParaView for visualization.")